In [1]:
import galprime as gp

from matplotlib import pyplot as plt

from astropy.io import fits
from astropy.table import Table

import numpy as np

from scipy.interpolate import interp1d


import warnings
warnings.filterwarnings('ignore')

In [2]:
test_fn = "gprime_out/tempfiles/100_0_0_0_0_done.pkl"

obj = gp.load_object(test_fn)



FileNotFoundError: [Errno 2] No such file or directory: 'gprime_out/tempfiles/100_0_0_0_0_done.pkl'

In [ ]:
print(gp.gprime_stop_codes[obj.stop_code])



Finished process


In [2]:
def to_fits(gp_object, filename="test_fits.fits"):
    hdul = fits.HDUList()

    head = fits.Header()
    head["ID"] = obj.id
    cfg_head = gp.header_from_config(gp_object.config)
    cfg_head.update(gp_object.params)

    for param_name, param in cfg_head.items():
        head[param_name] = param

    hdul.append(fits.PrimaryHDU(header=head))
    hdul.append(fits.ImageHDU(data=gp_object.convolved_model, name="CONV MOD"))
    hdul.append(fits.ImageHDU(data=gp_object.bg, name="BG"))
    hdul.append(fits.ImageHDU(data=gp_object.bg_model, name="BG MOD"))
    
    # for label, isolist in zip(["MOD", "BGADDED", "BGSUB"], gp_object.isophote_lists):
    #     hdul.append(fits.BinTableHDU(data = isolist["ISOLIST"].to_table()))
    hdul.writeto(filename, overwrite=True)
to_fits(obj)

NameError: name 'obj' is not defined

In [3]:
output = gp.load_object("gprime_out/additional_data/11121111_1_2_1.pkl")
config = gp.read_config_file("myconfig.gprime")

run_id = 11121111
config["RUN_ID"] = run_id
bin_id = "0_0_0"
outdirs = gp.gen_filestructure(config["OUTDIR"])
for key, val in outdirs.items():
    print(key, "\t", val)

MODEL_PROFS 	 gprime_out/model_profiles/
COADD_PROFS 	 gprime_out/coadd_profiles/
BGSUB_PROFS 	 gprime_out/bgsub_profiles/
MEDIANS 	 gprime_out/medians/
ADDL_DATA 	 gprime_out/additional_data/
TEMP 	 gprime_out/tempfiles/


In [5]:
gp.handle_output(output, outdirs, config, bin_id=bin_id)

In [ ]:

def highest_sma(profile_list):
    return np.min([max(profile.sma) for profile in profile_list])


def profiles_norm_sma(profile_list):
    sma_max = highest_sma(profile_list)
    sma_interp = np.arange(1, sma_max, 2)
    profile_interps = [interp1d(profile.sma, profile.intens, kind="linear", 
                                fill_value=0, bounds_error=False,
                                ) for profile in profile_list]
    profiles_stacked = np.vstack([interp(sma_interp) for interp in profile_interps])
    return sma_interp, profiles_stacked


def _median(profile_stack):
    return np.median(profile_stack, axis=0)


def gen_median(profile_list):
    profile_stack = profiles_norm_sma(profile_list)[1]
    return _median(profile_stack)



def bootstrap_median(profile_list, n_bootstraps=10000):
    smas, profile_stack = profiles_norm_sma(profile_list)

    # plt.imshow(profile_stack, aspect="auto", vmin=-0.005, vmax=0.005)

    sample_indices = np.random.choice(profile_stack.shape[0], (n_bootstraps, profile_stack.shape[0]), replace=True)

    medians = np.vstack([_median(profile_stack[sample]) for sample in sample_indices])

    sorted = np.sort(medians, axis=0)

    lower_index_1sig, upper_index_1sig = int(n_bootstraps * 0.159), int(n_bootstraps * 0.841)
    lower_index_2sig, upper_index_2sig = int(n_bootstraps * 0.023), int(n_bootstraps * 0.977)
    lower_index_3sig, upper_index_3sig = int(n_bootstraps * 0.002), int(n_bootstraps * 0.998)

    lower_1sig, upper_1sig = sorted[lower_index_1sig], sorted[upper_index_1sig]
    lower_2sig, upper_2sig = sorted[lower_index_2sig], sorted[upper_index_2sig]
    lower_3sig, upper_3sig = sorted[lower_index_3sig], sorted[upper_index_3sig]

    upper = np.vstack([upper_1sig, upper_2sig, upper_3sig])
    lower = np.vstack([lower_1sig, lower_2sig, lower_3sig])

    return smas, _median(profile_stack), lower, upper

bare_smas, bare_median, bare_low, bare_up = bootstrap_median(bare_profiles)
coadd_smas, coadd_median, coadd_low, coadd_up = bootstrap_median(coadd_profiles)
bgsub_smas, bgsub_median, bgsub_low, bgsub_up = bootstrap_median(bgsub_profiles)
